In [27]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
# import xgboost
from sklearn.ensemble import RandomForestRegressor 
# import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import minmax_scale,StandardScaler, MinMaxScaler
# from tqdm import tqdm_notebook

In [2]:
os.chdir(r"../Data/London")

In [3]:
df = pd.read_csv('AA_LONDON_DATA_VER3_DATA_TABLE.csv')

In [4]:
camdf = pd.read_csv('AA_LONDON_CAMERA_DATA_TABLE.csv')

In [5]:
os.chdir(r"../../Immediate Result")

In [6]:
df[:2]

,S_NO,CAMERA_NAME,DOWNLOAD_DATE,IMG_NAME,DENSITY_VALUE,DL_TIMESTAMP_LOCAL,COMMENTS,CAMERA_ID,DL_TIMESTAMP_CITY,SELECT_FLAG,RED,GREEN,BLUE
0,896208,549744,22-NOV-10,1290433594,16109.680,22-NOV-10 02.46.34.000000000 PM,NaN,182,22-NOV-10 01.46.34.000000000 PM,1,63621.507812,65366.414062,64108.488281
1,896209,549744,22-NOV-10,1290434044,11318.897,22-NOV-10 02.54.04.000000000 PM,NaN,182,22-NOV-10 01.54.04.000000000 PM,1,64379.992188,66520.500000,65209.515625


In [7]:
camdf[:2]

,CAMERA_ID,CAMERA_NAME,CAMERA_DESC,EASTING,NORTHING,GEO_LAT,GEO_LON,CONGESTION_ZONE
0,105,546600,A4 Cromwell Road by Earls Court Road,525351.39,178843.06,51.494572,-0.195707,0
1,106,546610,A4 Great West Road By Macbeth Street,522781.00,178369.00,51.490875,-0.232884,0


In [8]:
mergedf = df.merge(camdf,left_on="CAMERA_ID",right_on="CAMERA_ID")

tdf = mergedf[["CAMERA_ID","GEO_LON","GEO_LAT","DL_TIMESTAMP_CITY","DENSITY_VALUE"]].copy()

tdf['TIMESTAMP']  = pd.to_datetime(tdf['DL_TIMESTAMP_CITY'],format="%d-%b-%y %I.%M.%S.000000000 %p")

try:
    tdf.drop("DL_TIMESTAMP_CITY",axis=1,inplace=True)
except:
    print("column doesn't exist")

In [9]:
tdf[:2]

,CAMERA_ID,GEO_LON,GEO_LAT,DENSITY_VALUE,TIMESTAMP
0,182,-0.245122,51.616876,16109.680,2010-11-22 13:46:34
1,182,-0.245122,51.616876,11318.897,2010-11-22 13:54:04


In [10]:
tdf["TIME_EPOCH"] = (tdf.TIMESTAMP.astype(np.int64) // 10**9)

In [11]:
tdf["TIME_EPOCH"] -= tdf.TIME_EPOCH.min()

In [12]:
tdf.TIMESTAMP.describe()

count                  896246
unique                 401908
top       2010-10-24 10:17:03
freq                      163
first     2010-10-11 09:55:01
last      2010-11-22 18:59:42
Name: TIMESTAMP, dtype: object

In [13]:
tdf["TIME_BUCKET"] = tdf.TIME_EPOCH//600

In [14]:
tdf[:2]

,CAMERA_ID,GEO_LON,GEO_LAT,DENSITY_VALUE,TIMESTAMP,TIME_EPOCH,TIME_BUCKET
0,182,-0.245122,51.616876,16109.680,2010-11-22 13:46:34,3642693,6071
1,182,-0.245122,51.616876,11318.897,2010-11-22 13:54:04,3643143,6071


There are 89 cameras with more than 5000 entries

In [23]:
count_per_camera = tdf.groupby("CAMERA_ID").count().iloc[:,1]
cam_id_with_morethan_5000 = count_per_camera.index[count_per_camera > 5000]

In [24]:
tdf_1 = tdf[tdf.CAMERA_ID.isin(cam_id_with_morethan_5000)]

In [26]:
tdf_1[:2]

,CAMERA_ID,GEO_LON,GEO_LAT,DENSITY_VALUE,TIMESTAMP,TIME_EPOCH,TIME_BUCKET
214616,120,-0.126116,51.515619,20771.134,2010-11-22 12:56:01,3639660,6066
214617,120,-0.126116,51.515619,25959.132,2010-11-22 13:02:01,3640020,6066


aggregating records for each 30minutes for each camera:

In [30]:
tdf_30min = tdf_1.groupby(["CAMERA_ID",pd.Grouper(key="TIMESTAMP",freq='30min')]).mean()

Let's get the first and last recorded timestamp for each camera.

In [49]:
first_last_ts = tdf_30min.groupby("CAMERA_ID").apply(lambda x: (x.index[0][1],x.index[len(x)-1][1] ) )

Let's get most frequent start and end times

In [56]:
from collections import Counter

In [65]:
most_common_st = Counter([ts1 for ts1,ts2 in first_last_ts.values]).most_common(2)

most_common_en = Counter([ts2 for ts1,ts2 in first_last_ts.values]).most_common(2)

Finding cameras that have the same start and end times as most freq

In [101]:
cameras_with_most_freq_st_and_en_filter = tdf_30min.reset_index(level=1).groupby("CAMERA_ID").apply(lambda x: x.TIMESTAMP.min() == most_common_st[0][0] and x.TIMESTAMP.max() == most_common_en[0][0])
cameras_with_most_freq_st_and_en = cameras_with_most_freq_st_and_en_filter[cameras_with_most_freq_st_and_en_filter == True]

In [109]:
tdf_30min_matching_st_en = tdf_30min[tdf_30min.index.get_level_values(0).isin(cameras_with_most_freq_st_and_en.index)]

In [115]:
tdf_30min_matching_st_en.reset_index(1,inplace=True)

Let's select only the records between 9:30am and 6:30pm.

In [155]:
tdf_st_to_en = tdf_30min_matching_st_en[tdf_30min_matching_st_en.TIMESTAMP.apply(lambda x: x.time() >= most_common_st[0][0].time() and x.time() <= most_common_en[0][0].time()  )]

In [157]:
len(tdf_st_to_en)

45278

there are still some missing 30min bins:

In [156]:
tdf_st_to_en.groupby("CAMERA_ID").count()[:3]

,TIMESTAMP,GEO_LON,GEO_LAT,DENSITY_VALUE,TIME_EPOCH,TIME_BUCKET
CAMERA_ID,,,,,,
2,624,624,624,624,624,624
4,630,630,630,630,630,630
5,634,634,634,634,634,634


Let's add all the missing times with NaN values

In [168]:
all_daily_bins = pd.date_range(str(most_common_st[0][0].time()), str(most_common_en[0][0].time()), freq="30min").time

In [169]:
all_daily_bins 

array([datetime.time(9, 30), datetime.time(10, 0), datetime.time(10, 30),
       datetime.time(11, 0), datetime.time(11, 30), datetime.time(12, 0),
       datetime.time(12, 30), datetime.time(13, 0), datetime.time(13, 30),
       datetime.time(14, 0), datetime.time(14, 30), datetime.time(15, 0),
       datetime.time(15, 30), datetime.time(16, 0), datetime.time(16, 30),
       datetime.time(17, 0), datetime.time(17, 30), datetime.time(18, 0),
       datetime.time(18, 30)], dtype=object)

In [177]:
all_bins_series = pd.Series(all_daily_bins,name="TIMESTAMP")

Let's for each day, have 9:30 to 18:30 (with NaNs for missing)

In [308]:
def fill_with_na_per_day(df):
    st = df.TIMESTAMP.min()
    st = st.replace(hour=9,minute=30)
    en = df.TIMESTAMP.max()
    en = en.replace(hour=18,minute=30)
    rng = pd.date_range(st, en, freq="30min")
    df = df.merge(pd.Series(rng,name="TIMESTAMP"),how="outer")
    return df.sort_values("TIMESTAMP")
#     return .reset_index(level=1,drop=True)

In [309]:
tdf_st_en_with_nan = tdf_st_to_en.groupby(["CAMERA_ID",pd.Grouper(key="TIMESTAMP",freq='1D')]).apply(fill_with_na_per_day)

In [310]:
tdf_st_en_with_nan.reset_index(level=1,drop=True,inplace=True)

In [311]:
tdf_st_en_with_nan.reset_index(level=1,drop=True,inplace=True)

In [312]:
tdf_st_en_with_nan

,TIMESTAMP,GEO_LON,GEO_LAT,DENSITY_VALUE,TIME_EPOCH,TIME_BUCKET
CAMERA_ID,,,,,,
2,2010-10-11 09:30:00,-0.223275,51.490012,49826.607500,6.000000e+01,0.000000
2,2010-10-11 10:00:00,-0.223275,51.490012,37574.399333,1.180000e+03,1.555556
2,2010-10-11 10:30:00,-0.223275,51.490012,47599.954333,3.150000e+03,4.833333
2,2010-10-11 11:00:00,-0.223275,51.490012,42265.094556,4.800000e+03,7.555556
2,2010-10-11 11:30:00,-0.223275,51.490012,30107.977000,6.510300e+03,10.400000
2,2010-10-11 12:00:00,-0.223275,51.490012,18069.028100,8.311000e+03,13.400000
2,2010-10-11 12:30:00,-0.223275,51.490012,19753.544500,1.011100e+04,16.400000
2,2010-10-11 13:00:00,-0.223275,51.490012,21548.734400,1.191100e+04,19.400000
2,2010-10-11 13:30:00,-0.223275,51.490012,22430.078000,1.371100e+04,22.400000


Let's get the cameras that have all the 40 days.

In [313]:
countdf = tdf_st_en_with_nan.fillna(-1).groupby(["CAMERA_ID",pd.Grouper(key="TIMESTAMP",freq='1D')]).count()
daycountdf = countdf["DENSITY_VALUE"].groupby("CAMERA_ID").count()
cam_ids_with_all_40_days = daycountdf[daycountdf == 40].index

In [314]:
len(tdf_st_en_with_nan)

55765

In [317]:
cleandf_with_na = tdf_st_en_with_nan[tdf_st_en_with_nan.index.isin(cam_ids_with_all_40_days)]

In [318]:
len(cleandf_with_na)

44080

In [316]:
58*19*40 #58 cameras, 19 slots per day (30 min), 40 days.

44080

In [319]:
cleandf_with_na

,TIMESTAMP,GEO_LON,GEO_LAT,DENSITY_VALUE,TIME_EPOCH,TIME_BUCKET
CAMERA_ID,,,,,,
4,2010-10-11 09:30:00,-0.227278,51.491661,53547.356500,6.000000e+01,0.000000
4,2010-10-11 10:00:00,-0.227278,51.491661,46199.700875,1.290000e+03,1.750000
4,2010-10-11 10:30:00,-0.227278,51.491661,39602.705900,2.910000e+03,4.400000
4,2010-10-11 11:00:00,-0.227278,51.491661,27179.430333,4.800000e+03,7.555556
4,2010-10-11 11:30:00,-0.227278,51.491661,23527.222300,6.510100e+03,10.400000
4,2010-10-11 12:00:00,-0.227278,51.491661,19940.461500,8.311000e+03,13.400000
4,2010-10-11 12:30:00,-0.227278,51.491661,20405.220000,1.011100e+04,16.400000
4,2010-10-11 13:00:00,-0.227278,51.491661,21775.247000,1.191100e+04,19.400000
4,2010-10-11 13:30:00,-0.227278,51.491661,18727.309900,1.371100e+04,22.400000


Let's linearly interpolate the missing values!

In [321]:
cleandf = cleandf_with_na.interpolate(method="linear")

In [326]:
cleandf = cleandf.drop(["TIME_EPOCH","TIME_BUCKET"],axis=1)

In [331]:
cleandf.to_csv(r"cleandf_40days_930_to_1830_lin_interpolated.csv")